In [ ]:
#import packages
import time
import numpy as np
import capytaine as cpt
import scipy
from capytaine.io.mesh_writers import write_STL
import matplotlib.pyplot as plt
from scipy.linalg import block_diag
import vtk
import logging
import xarray as xr
from capytaine.io.xarray import merge_complex_values
from capytaine.post_pro import rao
logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

In [ ]:
# Import custom package
import DM_Assemble as DM_A
import DM_Reading  as DM_R
import DM_ShowNodes as DMshow
import DM_Verify as DM_V

In [ ]:
# Show nodes
file_path = "Job-1_largemesh.inp"
label_interval = 1
DMshow.plot_fea_model(file_path, label_interval)

In [ ]:
# Example usage:
DM_V.verify_hydrodynamic_data("data\DMFEM1010_180_direction45.nc",wave_direction=np.pi/4)

In [ ]:
# main 
if __name__ == "__main__":
    N = 793   # Suppose the number of nodes is 793 63

    # Obtain the dataset
    dataset = merge_complex_values(xr.open_dataset("BM10_180_direction0.nc"))
    
    # Extract matrices and force from the dataset
    added_mass = dataset['added_mass'][0].values
    radiation_damping = dataset['radiation_damping'][0].values
    inertia_matrix = dataset['inertia_matrix'].values
    hydrostatic_stiffness = dataset['hydrostatic_stiffness'].values
    F_w = dataset['Froude_Krylov_force'][0].values + dataset['diffraction_force'][0].values
    
    # Construct the combined matrices
    M = added_mass + inertia_matrix  # Total mass
    C = radiation_damping  # Damping
    K = hydrostatic_stiffness  # Stiffness
    
    # Sample node ID list
    # node_ids = [41, 39, 37, 35, 33, 31, 29, 27, 25, 23]
    node_ids = DM_A.calculate_node_positions(425,6,10)  # Calculate the node positions
    # print(node_ids)
    # Call function to insert the additional mass stiffness matrix into the total stiffness matrix
    mass = DM_A.insert_martix(N, M, node_ids)
    damping = DM_A.insert_martix(N, C, node_ids)
    hy_stiffness = DM_A.insert_martix(N, K, node_ids)

    # Obtain the structural stiffness matrix
    file_path = "Job-1-frec_STIF1.mtx"  # Replace with your file path
    stiffness = DM_R.get_stiffness_matrix(file_path) + hy_stiffness

    # Assemble the force matrix
    K_F_w = DM_A.extend_force_matrix(F_w, node_ids, N)
    
    omega = 0.5754  # Here we use 1.0 as an example, in actual applications you should use your real data
    X = DM_A.solve_frequency_domain(mass, damping, stiffness, K_F_w, omega)  # Solve in frequency domain

    # Calculate internal forces
    st = stiffness-hy_stiffness
    internal_forces = np.dot(st, X)



In [ ]:
# Import force analysis model
from DM_forec_analysis import ForceAnalysis
# Create an instance of the ForceAnalysis class
# Parameters include:
# N: Total Number of nodes
# H: Row
# module_nodes: Number of nodes per module
# module_number: Number of modules
# element_length: Length of the element
# element_width: Width of the element
analyzer = ForceAnalysis(N=793, H=13, module_nodes=7, module_number=10, element_length=5, element_width=5)

# Compute module displacements using the provided displacement matrix X
module_displacements = analyzer.compute_module_displacements(displacement_matrix=X)

# Fetch the stiffness matrix for an element from a specified file
K_element = DM_R.get_stiffness_matrix("data\Job-1-element55_STIF1.mtx")

# Compute forces within the module using the stiffness matrix and module displacements
module_forces = analyzer.compute_module_forces(K_element, module_displacements)

# Map the forces from module level to global nodes
global_forces = analyzer.map_forces_to_global_nodes(module_forces)

# Visualize global forces using a plot
analyzer.plot_forces(global_forces)

# Display global forces as a 2D heatmap
analyzer.plot_2D_heatmap(global_forces)

# Display global forces as a 3D surface plot
analyzer.plot_3D_surface(global_forces)

In [ ]:
# Show nodes
file_path = "data\Job-1_sub.inp"
label_interval = 1
DMshow.plot_fea_model(file_path, label_interval)

In [ ]:
# Show heave displacement
import matplotlib.pyplot as plt
displacement = X
# 提取中轴线上的节点位移
mid_axis_nodes_start = 367  # 起始节点
mid_axis_nodes_end = 427 # 结束节点

# 由于每个节点有6个自由度，我们需要找到与这些节点对应的位移
mid_axis_displacement = displacement[(mid_axis_nodes_start-1)*6:(mid_axis_nodes_end)*6:6, :]

# 垂荡位移应该是Y轴（第2个自由度）的位移，我们需要每6个自由度取一次
heave_displacement = displacement[(mid_axis_nodes_start-1)*6+2:(mid_axis_nodes_end)*6:6, :]

# 定义节点在长度方向上的位置
length = np.linspace(0, 300, mid_axis_nodes_end-mid_axis_nodes_start+1)

# 绘制位移曲线
plt.figure(figsize=(10, 6))
plt.plot(length, np.abs(heave_displacement), marker='o')  # 绘制垂荡位移的绝对值
plt.xlabel("Length (m)", fontsize=12)
plt.ylabel("Heave displacement", fontsize=12)
plt.title("Heave displacement along the length", fontsize=14)
plt.grid(True)
plt.show()
